---
# Sieci neuronowe - projekt 3

---
## Załadowanie bibliotek

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from enum import Enum

---
## Załadowanie danych

In [2]:
temperature_data = pd.read_csv('temperature.csv')
wind_data = pd.read_csv('wind_speed.csv')
temperature_data = temperature_data.fillna(temperature_data.mean())
wind_data = wind_data.fillna(wind_data.mean())

temperature_data.head(5)

C:\Users\jkobo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jkobo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01 12:00:00,283.862654,284.992929,288.155821,284.409626,290.846116,290.215044,292.424887,295.493358,285.617856,...,285.374168,285.400406,280.343010,283.779823,291.521986,294.512307,309.100000,295.266398,294.094803,293.184253
1,2012-10-01 13:00:00,284.630000,282.080000,289.480000,281.800000,291.870000,291.530000,293.410000,296.600000,285.120000,...,285.630000,288.220000,285.830000,287.170000,307.590000,305.470000,310.580000,304.400000,304.400000,303.500000
2,2012-10-01 14:00:00,284.629041,282.083252,289.474993,281.797217,291.868186,291.533501,293.403141,296.608509,285.154558,...,285.663208,288.247676,285.834650,287.186092,307.590000,304.310000,310.495769,304.400000,304.400000,303.500000
3,2012-10-01 15:00:00,284.626998,282.091866,289.460618,281.789833,291.862844,291.543355,293.392177,296.631487,285.233952,...,285.756824,288.326940,285.847790,287.231672,307.391513,304.281841,310.411538,304.400000,304.400000,303.500000
4,2012-10-01 16:00:00,284.624955,282.100481,289.446243,281.782449,291.857503,291.553209,293.381213,296.654466,285.313345,...,285.850440,288.406203,285.860929,287.277251,307.145200,304.238015,310.327308,304.400000,304.400000,303.500000


---
## Przetworzenie danych

In [3]:
def calculate_means_by_days(data):
    data['datetime'] = pd.to_datetime(data['datetime'])
    data['date'] = data['datetime'].dt.date
    data = data.drop('datetime', axis=1)
    data = data.groupby('date').mean().reset_index()
    data = data[['date'] + [col for col in data.columns if col != 'date']]

    return data

temperature_data = calculate_means_by_days(temperature_data)
wind_data = calculate_means_by_days(wind_data)

In [4]:
temperature_data.head(5)

,date,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
0,2012-10-01,284.557593,282.357758,289.311574,281.987459,291.763135,291.460291,293.281076,296.601041,285.488012,...,285.987415,288.305335,285.425899,287.071818,305.363194,303.437593,310.070609,303.638867,303.541234,302.640354
1,2012-10-02,286.145190,286.137728,292.958306,285.156888,295.890450,295.291472,297.248385,301.211968,289.771821,...,289.239595,290.892389,286.937931,289.013090,302.226773,302.787467,306.759071,303.900000,303.900000,302.675000
2,2012-10-03,285.528125,289.599792,296.929167,287.673958,299.008542,297.878750,300.691875,302.867083,291.205417,...,290.353542,290.065625,287.374583,289.020833,301.194375,301.687917,303.289583,301.561042,301.502500,301.258125
3,2012-10-04,284.373333,286.482500,295.687083,284.391667,295.997917,296.080833,301.820000,302.232917,293.096250,...,293.633750,291.987083,286.860833,290.043750,300.094167,299.940000,301.770208,299.139167,299.139167,298.924167
4,2012-10-05,283.757292,288.286042,290.635417,284.756250,292.948333,293.894375,300.628542,301.811250,292.829167,...,294.015833,294.043542,287.535208,289.517292,299.712083,300.153125,299.860000,298.877500,298.877500,297.547500


---
## Funkcje pomocnicze

In [5]:
def DrawData(X, y, plot_name):
    plt.style.use('dark_background')
    plt.figure(figsize=(10,5))
    axes = plt.gca()
    axes.set(xlabel="$X_1$", ylabel="$X_2$")
    plt.title(plot_name, fontsize=30)
    #plt.subplots_adjust(left=0.20)
    #plt.subplots_adjust(right=0.80)

    plt.scatter(X[0, :], X[1, :], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')

def DrawDataCompare(X, y, y_pred):
    plt.style.use('dark_background')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    ax1.set(xlabel="$X_1$", ylabel="$X_2$")
    ax1.set_title('Actual labels', fontsize=20)
    ax1.scatter(X[0, :], X[1, :], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    
    ax2.set(xlabel="$X_1$", ylabel="$X_2$")
    ax2.set_title('Predicted labels', fontsize=20)
    ax2.scatter(X[0, :], X[1, :], c=y_pred.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    
    plt.tight_layout()
    plt.show()

def GetClassificationData(name):
    file = pd.read_csv(name, sep=",")

    input = np.array(file[["x", "y"]])
    results = np.array(file["cls"] - 1)

    num_classes = results.max() + 1
    
    return input.T, results.T, num_classes

def get_data_for_city(data, city, part):
    n = len(data)
    ind = int(n*part)
    data_train = data[:ind][city].tolist()
    data_test = data[ind+1:][city].tolist()
    return data_train, data_test

def get_windows(data):
    n = len(data)
    windows_X = []
    windows_y = []

    for i in range(n - 4):
        window = data[i:i+3]
        windows_X.append(window)
        windows_y.append(data[i+4])

    return windows_X, windows_y

def convert_range(data, range_min, range_max):
    old_min = data.min()
    old_max = data.max()
    return (((data - old_min) * (range_max - range_min)) / (old_max - old_min)) + range_min


---
## Funkcje aktywacji

In [6]:
class Sigmoid():
    @staticmethod
    def calculate(x):
        x = np.clip( x, -500, 500 )
        return 1 / (1 + np.exp(-x))

    def calculateDeriv(self, x):
        return x * (1 - x)

    
class ReLU():
    @staticmethod
    def calculate(x):
        return np.maximum(x, 0)
    
    @staticmethod
    def calculateDeriv(x):
        return x > 0
    
    
class CrossEntropy():
    @staticmethod
    def calculate(y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return - (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred)).mean()

    @staticmethod
    def calculateDeriv(y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return - (y / y_pred) + (1 - y) / (1 - y_pred)
    
class Softmax():
    @staticmethod
    def calculate(x):
        x = np.clip(x, 1e-15, 1 - 1e-15)
        e_x = np.exp(x - np.max(x, axis=0, keepdims=True))
        return e_x / np.sum(e_x, axis=0, keepdims=True)
    
    def calculateDeriv(self, x):
        value = self.calculate(x)
        return value * (1 - value)
    
class MSE:
    @staticmethod
    def calculate(y, y_pred):
        return ((y - y_pred) ** 2).mean()
    
    @staticmethod
    def calculateDeriv(y, y_pred):
        return -2*(y - y_pred) / y.shape[0]
    
class MAE:
    @staticmethod
    def calculate(y, y_pred):
        n = len(y)
        return np.sum(np.abs(y- y_pred)) / n
    
    @staticmethod
    def calculateDeriv(y, y_pred):
        n = len(y)
        return np.sign(y_pred - y) / n
        

class Tanh:
    @staticmethod
    def calculate(x):
        return np.tanh(x)

    @staticmethod
    def calculateDeriv(x):
        return 1 - x ** 2
    
class Linear:
    @staticmethod
    def calculate(x):
        return x

    @staticmethod
    def calculateDeriv(x):
        return np.ones_like(x)

---
## Sieć neuronowa

In [7]:
class NeuralNetworkStructure:
    def __init__(self, inputSize, outputSize, hiddenLayerSizes, hiddenLayerFunction, outputLayerFunction):
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenLayerSizes = hiddenLayerSizes
        self.layersSizes = hiddenLayerSizes + [outputSize]
        self.activationFunction = [hiddenLayerFunction] * len(hiddenLayerSizes) + [outputLayerFunction]
        self.layerInput = [None] * len(self.layersSizes)
        self.layerOutput = [None] * len(self.layersSizes)

        self.initializeWeights()

    def initializeWeights(self):
        self.weights = []
        self.bias = []
        
        previousLayerSize = self.inputSize
        for layerSize in self.layersSizes:
            self.weights.append(np.random.rand(layerSize, previousLayerSize) - 0.5)
            self.bias.append(np.random.rand(layerSize, 1) - 0.5)
            previousLayerSize = layerSize
            

class NeuralNetwork:
    def __init__(self, neuralNetworkStructure, epochs, learningRate, lossFunction, partsCount = 1):
        self.structure = neuralNetworkStructure
        self.lossFunction = lossFunction
        self.learningRate = learningRate
        self.epochs = epochs
        self.partsCount = partsCount

    def Forward(self, X):
        previous_layer = X #np.reshape(X, (X.shape[0], 1))
        for id in range(len(self.structure.layersSizes)):
            self.structure.layerInput[id] = self.structure.weights[id].dot(previous_layer) + self.structure.bias[id]
            self.structure.layerOutput[id] = self.structure.activationFunction[id].calculate(self.structure.layerInput[id])
            previous_layer = self.structure.layerOutput[id]
        return previous_layer
    
    
    def Backward(self, X, ExpectedY, PredictedY):
        ExpectedY = np.reshape(ExpectedY, (1, -1))
        previous_layer_error = self.lossFunction.calculateDeriv(ExpectedY, PredictedY)
        
        for id in range(len(self.structure.layersSizes) -1, -1, -1):    
            previous_layer_output = self.structure.layerOutput[id - 1] if id != 0 else X
            
            delta = previous_layer_error * self.structure.activationFunction[id].calculateDeriv(self.structure.layerOutput[id])
            deltaW = np.dot(delta, previous_layer_output.T) 
            deltaB = np.sum(delta, axis=1, keepdims=True)
            previous_layer_error = np.dot(self.structure.weights[id].T, delta)
                                        
            self.structure.weights[id] -= self.learningRate * deltaW / ExpectedY.shape[1]
            self.structure.bias[id] -= self.learningRate * deltaB / ExpectedY.shape[1]
    
    def one_hot(self, Y):
        one_hot_Y = np.zeros((Y.max() + 1, Y.size))
        one_hot_Y[Y, np.arange(Y.size)] = 1
        return one_hot_Y
    
    def Train(self, X, ExpectedY):
    
        #X = np.reshape(X, (-1, 1))
        #ExpectedY = np.reshape(ExpectedY, (-1, 1))
    
        predictedY = self.Forward(X)
        self.Backward(X, ExpectedY, predictedY)

    def Test(self, train_inputs, train_results, test_inputs, test_results, testAfterEpochs = 10, schuffleParts = True, drawData = False, saveWeights = False):
        one_hot_results = self.one_hot(train_results)
        acc = []
        epochs = []

        for epoch in range(self.epochs):
            inputParts = np.array_split(train_inputs, self.partsCount, 1)
            resultsParts = np.array_split(one_hot_results, self.partsCount, 1)
            parts_range = list(range(len(inputParts)))
            
            if schuffleParts == True:
                np.random.shuffle(parts_range)
            
            for i in parts_range:
                self.Train(inputParts[i], resultsParts[i])
            
            if (epoch + 1) % testAfterEpochs == 0:        
                predictedY = self.Forward(test_inputs)
                if drawData == True:
                    DrawDataCompare(test_inputs, test_results, np.argmax(predictedY, axis=0))
                if saveWeights == True:
                    fig = plt.figure(figsize = (20, 12)) # width x height
                    for i in range(len(self.structure.weights)):
                        ax1 = fig.add_subplot(3, 5, i+1) # row, column, position
                        sns.heatmap(self.structure.weights[i], cmap="coolwarm", ax=ax1) # jaka paleta? moze Spectral albo magma?
                        plt.title(f'Weight Heatmap - Layers {i}-{i+1}')
                        mean = np.mean(self.structure.weights[i])
                        std = np.std(self.structure.weights[i])
                        print(f'Layers {i}-{i+1}: Mean={mean}, Std={std}')
                    plt.savefig(f'plots/e{epoch+1}')
                    #plt.show()
                    plt.close(fig)
                correct = np.sum(np.argmax(predictedY, 0) == test_results)
                
                test_accuracy = correct / len(test_results)
                print(f"Epoch {epoch + 1}/{self.epochs}, Test Accuracy: {test_accuracy * 100:.2f}% Correct: {correct}, All: {len(test_results)} ")
                epochs.append(epoch+1)
                acc.append(test_accuracy)
        return epochs, acc
    
    
    
    def TestRegression(self, train_inputs, train_results, test_inputs, schuffleParts = True):
        prediction = []
        train_inputs = np.array(train_inputs).T
        train_results = np.array(train_results).T
        test_inputs = np.array(test_inputs).T
        for epoch in range(self.epochs):
            
            inputParts = np.array_split(train_inputs, self.partsCount, 0)
            resultsParts = np.array_split(train_results, self.partsCount, 0)
            parts_range = list(range(len(inputParts)))
            
            if schuffleParts == True:
                np.random.shuffle(parts_range)
        

            for i in parts_range:
                self.Train(train_inputs, train_results)
            
            prediction = self.Forward(test_inputs)
            
            
        return prediction   
    
    def Predict(self, test_inputs):
        predictedY = self.Forward(test_inputs)
        return predictedY


In [34]:
data_train, data_test = get_data_for_city(temperature_data, 'Vancouver', 0.3)

train_windows_X, train_windows_y = get_windows(data_train)
test_windows_X, test_windows_y = get_windows(data_test)
train_windows_X
temperature_data['Vancouver'].head(10)

0    284.557593
1    286.145190
2    285.528125
3    284.373333
4    283.757292
5    284.825833
6    285.595833
7    285.943333
8    285.957292
9    284.913958
Name: Vancouver, dtype: float64

In [42]:
import numpy as np
import pandas as pd


nnS = NeuralNetworkStructure(
    inputSize = 3, 
    outputSize = 1, 
    hiddenLayerSizes = [8, 4], 
    hiddenLayerFunction = Sigmoid(), 
    outputLayerFunction = Linear())

nn = NeuralNetwork(  
    epochs = 3000, 
    learningRate = 0.1,
    neuralNetworkStructure = nnS,
    lossFunction = MAE())

predictedPoints = nn.TestRegression(train_windows_X, train_windows_y, test_windows_X)
predictedPoints
err = abs(np.array(test_windows_y) - np.array(predictedPoints))
predictedPoints
print(f'Number of predictions with error > 2: {np.sum(err > 2)}/{err.shape[1]}')


Number of predictions with error > 2: 1036/1316
